PROCESAMIENTO DE LENGUAGE NATURAL
===
ANALISIS DE TÓPICOS
Latent Dirichlet allocation (LDA)
Por: Alexander Siavichay

Latent Dirichlet allocation (LDA), permite clasificar textos contenidos en un documento en tópicos. Construye el tópico desde un modelo de documento y palabras desde modelos de tópicos 

Se utiliza la librería gensim y spacy, "spaCy es una biblioteca para el procesamiento avanzado de lenguaje natural en Python y Cython".

Previamente se realizó un trabajo de análsis de texto con gensim, sin embargo, se vió que los resultados en español no eran del todo efectivos.

En consecuencia, se propuso realizar un análisis de tópicos de un canal de noticias ecuatoriano muy conocido, Ecuavisa. A partir de un tratamiento html de su página web, se pudo obtener con un get con 16000 noticias para ser analizadas.



## ANTECEDENTES

Ecuavisa es un canal que tiene entre sus espacios televisivos, uno  dedicado exclusivamente al reporte de noticias. En su pagina web no se evidencia una clasificacion de las noticias que se exponen, por ello, en nuestro caso haremos una clasificacion utilizando herramientas de procesamiento de lenguage natural

Para poder orientar los resultados, se analizó otros medios que hacen una clasificación, resultando en 
- 1. Política
- 2. Economía
- 3. Internacional
- 4. Nacional
- 5. Seguridad
- 6. Actualidad
- 7. Crónica
- 8. Deportes

## Obtención del corpus de datos


In [37]:

import requests
from bs4 import BeautifulSoup

#Tomamos 5232 noticias de ecuavisa, cada enlace tiene 16 noticias de los 326 enlaces disponibles
i=0
enlaces=[]
url="http://ecuavisa.com/historico/noticias/nacional?page="
while i<=326:
    r=requests.get(url+str(i))
    enlaces.append(r)
    i+=1


## Almacenamiento del corpus obtenido en un svc

In [38]:
from sklearn import svm
from sklearn import datasets
data = svm.SVC()
#Vectorizamos cada noticia en los 100 enlaces
noticias=[]

for enlace in enlaces:
    soup = BeautifulSoup(enlace.content, 'html.parser')
    contenido=soup.find_all("span",class_="field-content")
    for noticia in contenido:
        texto=noticia.find("a").get_text().strip()
        noticias.append((texto))
        
print(len(noticias)," noticias encontradas")
print(noticias[0])

5232  noticias encontradas
Feriados de 2020: Ecuador tendrá 27 días de descanso


In [39]:
#Guardamos los resultados en un svc
import numpy

numpy.savetxt("ecuavisa.csv", noticias, fmt="%s")

## Carga de datos con pandas

In [40]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('ecuavisa.csv',encoding = "ISO-8859-1",header=None,usecols=[0], names=['Noticia']);
data_text = data;#[['Noticia']]; 
print(data_text)
# We only need the Headlines text column from the data
#data_text = data[['headline_text']];

                                                Noticia
0     Feriados de 2020: Ecuador tendrá 27 días de de...
1     Piden declarar héroe a niño que salvó a otros ...
2                                               Limpias
3     Personas que abusen sexualmente de menores no ...
4     Consumo de gasolina Súper se redujo un 40% en ...
5     ¿Por qué Odebrecht no ha sido procesada en Ecu...
6     Asamblea aprueba publicación de reformas tribu...
7     Deslizamiento en vía Alóag-Sto. Domingo afecta...
8     Hombre mata con una puñalada a exconviviente e...
9     Agreden a vigilante con machete en el cantón Y...
10                   Figuras de superhéroes y políticos
11    Guayaquil registró la precipitación más alta a...
12    Seis fallecidos por dos accidente al sur de Quito
13    Reportan récord de temperatura máxima en Guaya...
14    Dos sismos se registraron en el archipiélago d...
15    Presidente Moreno indulta a 8 personas privada...
16    Gobierno destaca leve reducción de desnutr

In [41]:
'''
Add an index column to the dataset and save the dataset as 'documents'
'''
data_text['index'] =data.index
documents = data_text
print (data_text)

                                                Noticia  index
0     Feriados de 2020: Ecuador tendrá 27 días de de...      0
1     Piden declarar héroe a niño que salvó a otros ...      1
2                                               Limpias      2
3     Personas que abusen sexualmente de menores no ...      3
4     Consumo de gasolina Súper se redujo un 40% en ...      4
5     ¿Por qué Odebrecht no ha sido procesada en Ecu...      5
6     Asamblea aprueba publicación de reformas tribu...      6
7     Deslizamiento en vía Alóag-Sto. Domingo afecta...      7
8     Hombre mata con una puñalada a exconviviente e...      8
9     Agreden a vigilante con machete en el cantón Y...      9
10                   Figuras de superhéroes y políticos     10
11    Guayaquil registró la precipitación más alta a...     11
12    Seis fallecidos por dos accidente al sur de Quito     12
13    Reportan récord de temperatura máxima en Guaya...     13
14    Dos sismos se registraron en el archipiélago d...

In [42]:
'''
Get the total number of documents
'''
print(len(documents))

5232


In [44]:
'''
Preview a document and assign the index value to 'document_num'
'''
document_num = 3000
print("\n**Printing out a sample document:**")
print(documents[documents['index'] == document_num])


**Printing out a sample document:**
                                                Noticia  index
3000  Se requieren $650 millones para reparar los 5 ...   3000


In [45]:
'''
Seperate the value of the headline from the document selected with 'document_num'
'''
documents.Noticia[document_num]

'Se requieren $650 millones para reparar los 5 proyectos petroleros analizados por ONU'

In [46]:
'''
Loading Gensim and nltk libraries
'''
#pip install gensim
import gensim
from gensim.utils import simple_preprocess
#from gensim.parsing.preprocessing import STOPWORDS
#from nltk.stem import WordNetLemmatizer, SnowballStemmer
#from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
#import nltk
#nltk.download()

In [47]:
#pip install -U spacy
import spacy
from spacy import displacy
from spacy.lang.es.stop_words import STOP_WORDS

In [48]:
'''
Lemmatizing example for a verb, noun.
'''
#Se necesita instalar python -m spacy download es_core_news_sm


nlp = spacy.load('es_core_news_sm')
doc = nlp("Quito enciende luces navideñas en su centro")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
    print(len(token.dep_))


Quito Quito NOUN NOUN__Gender=Masc|Number=Sing nsubj Xxxxx True False
5
enciende encender VERB VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin ROOT xxxx True False
4
luces luz NOUN NOUN__Gender=Fem|Number=Plur obj xxxx True False
3
navideñas navideño ADJ ADJ__Gender=Fem|Number=Plur amod xxxx True False
4
en en ADP ADP__AdpType=Prep case xx True True
4
su su DET DET__Number=Sing|Person=3|Poss=Yes|PronType=Prs det xx True True
3
centro centrar NOUN NOUN__Gender=Masc|Number=Sing obl xxxx True False
3


In [49]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def preprocessSpacy(doc):
    doc=nlp(doc)
    result=[]
    for token in doc:
        if not token.is_stop and len(token.text)>3:
            result.append(token.lemma_)
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            #    token.shape_, token.is_alpha, token.is_stop)
        
    return result

In [50]:
'''
Preview a document after preprocessing
'''

doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
#print(preprocess(doc_sample))
print(preprocessSpacy(doc_sample))


Original document: 
['Se', 'requieren', '$650', 'millones', 'para', 'reparar', 'los', '5', 'proyectos', 'petroleros', 'analizados', 'por', 'ONU']


Tokenized and lemmatized document: 
['requerir', 'millón', 'reparar', 'proyecto', 'petrolero', 'analizar']


In [51]:
'''
Save the values of the headlines into a variable 'training_headlines' from 'documents'
'''
training_headlines =documents.Noticia[:]

In [52]:
'''
Preview 'training_headlines'
'''
training_headlines

0       Feriados de 2020: Ecuador tendrá 27 días de de...
1       Piden declarar héroe a niño que salvó a otros ...
2                                                 Limpias
3       Personas que abusen sexualmente de menores no ...
4       Consumo de gasolina Súper se redujo un 40% en ...
5       ¿Por qué Odebrecht no ha sido procesada en Ecu...
6       Asamblea aprueba publicación de reformas tribu...
7       Deslizamiento en vía Alóag-Sto. Domingo afecta...
8       Hombre mata con una puñalada a exconviviente e...
9       Agreden a vigilante con machete en el cantón Y...
10                     Figuras de superhéroes y políticos
11      Guayaquil registró la precipitación más alta a...
12      Seis fallecidos por dos accidente al sur de Quito
13      Reportan récord de temperatura máxima en Guaya...
14      Dos sismos se registraron en el archipiélago d...
15      Presidente Moreno indulta a 8 personas privada...
16      Gobierno destaca leve reducción de desnutrició...
17      Un mue

In [53]:
processed_docs=documents['Noticia'].map(preprocessSpacy)

In [54]:
'''
Preview 'processed_docs'
'''
processed_docs

0                    [Feriados, 2020, Ecuador, descansar]
1          [Piden, declarar, héroe, niño, salvar, chocar]
2                                               [Limpias]
3                  [Personas, abusar, sexualmente, menor]
4              [Consumo, gasolina, Súper, reducir, mesar]
5                          [Odebrecht, procesar, Ecuador]
6       [Asamblea, aprobar, publicación, reformar, tri...
7       [Deslizamiento, Alóag, Domingo, afecto, vehículo]
8       [Hombre, matar, puñalada, exconviviente, Guaya...
9         [Agreden, vigilante, machete, cantón, Yaguachi]
10                       [Figuras, superhéroes, político]
11      [Guayaquil, registrar, precipitación, alto, ni...
12                         [fallecido, accidentar, Quito]
13      [Reportan, récord, temperatura, máximo, Guayaq...
14            [sismo, registrar, archipiélago, Galápagos]
15      [Presidente, Moreno, indultar, personar, priva...
16      [Gobierno, destacar, levar, reducción, desnutr...
17            

In [55]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears in the training set using gensim.corpora.Dictionary
and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [57]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

print(len(dictionary))

0 2020
1 Ecuador
2 Feriados
3 descansar
4 Piden
5 chocar
6 declarar
7 héroe
8 niño
9 salvar
10 Limpias
5548


In [61]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=5, no_above=0.1)
len(dictionary)

1243

In [62]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [63]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
# document_num = 4310
bow_corpus[document_num]

[(65, 1), (66, 1), (114, 1), (732, 1), (1075, 1)]

In [64]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 65 ("millón") appears 1 time.
Word 66 ("petrolero") appears 1 time.
Word 114 ("analizar") appears 1 time.
Word 732 ("proyecto") appears 1 time.
Word 1075 ("requerir") appears 1 time.


In [65]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [66]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf = tfidf[bow_corpus]

In [67]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.9222548025251416), (1, 0.3865825645567994)]


RESULTADOS
===

In [72]:
'''
Define lda model using tfidf corpus
'''

lda_model_tfidf = gensim.models.LdaMulticore(bow_corpus, 
                                    num_topics = 8, 
                                    id2word = dictionary,                                    
                                    passes = 100)

In [73]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.061*"herir" + 0.048*"accidentar" + 0.042*"dejar" + 0.035*"morir" + 0.030*"muerto" + 0.026*"detenido" + 0.021*"drogar" + 0.021*"tránsito" + 0.019*"incendiar" + 0.018*"fallecido"


Topic: 1 Word: 0.025*"millón" + 0.022*"iniciar" + 0.020*"Gobierno" + 0.019*"protestar" + 0.018*"reformar" + 0.018*"Asamblea" + 0.017*"Nacional" + 0.016*"pagar" + 0.016*"cerrar" + 0.016*"Corte"


Topic: 2 Word: 0.037*"personar" + 0.019*"procesar" + 0.018*"pedir" + 0.016*"casar" + 0.015*"consultar" + 0.015*"caer" + 0.014*"Consejo" + 0.014*"Judicatura" + 0.014*"nacional" + 0.012*"CPCCS"


Topic: 3 Word: 0.024*"Esmeraldas" + 0.022*"operativo" + 0.020*"Guayas" + 0.017*"juicio" + 0.017*"audiencia" + 0.015*"detenido" + 0.015*"político" + 0.014*"seguridad" + 0.014*"analizar" + 0.014*"pedir"


Topic: 4 Word: 0.156*"Ecuador" + 0.031*"asesinar" + 0.024*"venezolano" + 0.020*"año" + 0.016*"frontero" + 0.015*"niño" + 0.014*"muerte" + 0.012*"cárcel" + 0.010*"violación" + 0.010*"Cuenca"


Topic: 5 Word: 0.047

### ANÁLISIS
En los resultados anteriores tenemos los tópicos con sus conjuntos de palábras referenciales. Es momento de analizar cada tópico propuesto en los antecedentes y emparejar con los tópicos obtenidos

Tópicos propuestos:
- 1. Política
- 2. Economía
- 3. Internacional
- 4. Nacional
- 5. Seguridad
- 6. Actualidad
- 7. Crónica
- 8. Deportes

Emparejando con los tópicos obtenidos:
* Tópico 0: Crónica
* Tópico 1: Política
* Tópico 2: *Asamblea*
* Tópico 3: Seguridad
* Tópico 4: *Violencia*
* Tópico 5: *Corrupción*
* Tópico 6: *Actualidad en Quito*
* Tópico 7: *Policial*

Como se puede observar, no necesariamente se pudo obtener los tópicos que habíamos señalado en un inicio, referenciados desde otros noticieros. Aparecen otros temas, como el de Asamblea, Violencia, Corrupción, Policial, que no habíamos considerado pero que el algoritmo nos devuelve.

Con esto podríamos concluir que Ecuavisa, trata estos temas en su mayoría de noticias.

## CONCLUSIONES

Como se puede observar en los resultados, los tópicos encontrados deben finalmente ser analizados por una persona del área de conocimiento. De acuerdo a nuestra hipótesis coincidimos en un 50%, sin embargo falta por hacer un análisis más profundo.

El LDA, nos permite evidenciar ciertos aspectos en el análisis de textos que serían complicados de hacer con una lectura rápida de todos los temas tratados. Como recomendación, se debe tener mucho cuidado en la especificación de passes e iteraciones del algoritmo, al cambiar estos valores, los resultados varían significativamente.

## RERERENCIAS
https://www.edureka.co/community/42836/how-to-read-pandas-csv-file-with-no-header

https://medium.com/@yeralway1/primeros-pasos-en-nlp-con-spacy-un-vistazo-general-734686843a57

https://radimrehurek.com/gensim/models/ldamulticore.html